In [1]:
!conda env list

# conda environments:
#
base                     E:\Softwares\Anaconda
quantum               *  E:\Softwares\Anaconda\envs\quantum
                         E:\Softwares\Miniconda
                         E:\Softwares\Orange



In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score, pairwise_distances, silhouette_samples, silhouette_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans, AgglomerativeClustering 
from sklearn.manifold import TSNE

# classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections


# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

from time import time
from datetime import datetime
import matplotlib.cm as cm
from scipy.spatial.distance import cdist

from datetime import datetime
from joblib import Parallel, delayed

import seaborn as sns
import plotly.express as px
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
os.chdir("D:/Research/SC-ML/Model/best vif without sampling")

In [4]:
X = pd.read_csv("x_vif_only.csv")
X.drop(['Unnamed: 0','rev_stop'], axis=1, inplace=True)
y = pd.read_csv("y.csv")
df=pd.concat([X,y],axis=1)
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
X = pd.read_csv("x_vif_only.csv")
X.drop(['Unnamed: 0','rev_stop'], axis=1, inplace=True)
y = pd.read_csv("y.csv", index_col=None)['went_on_backorder']

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

print('No backorder', round(y.value_counts()[0]/len(y) * 100,2), '% of the dataset')
print('Backorder', round(y.value_counts()[1]/len(y) * 100,2), '% of the dataset')

sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]

# We already have X_train and y_train for undersample data thats why I am using original to distinguish and to not overwrite these variables.
# original_Xtrain, original_Xtest, original_ytrain, original_ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Turn into an array
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)
print('-' * 100)

print('Label Distributions: \n')
print(train_counts_label/ len(original_ytrain))
print(test_counts_label/ len(original_ytest))

No backorder 99.3 % of the dataset
Backorder 0.7 % of the dataset
Train: [ 259572  259620  259654 ... 1775315 1775316 1775317] Test: [     0      1      2 ... 355895 355896 355897]
Train: [      0       1       2 ... 1775315 1775316 1775317] Test: [259572 259620 259654 ... 711229 711230 711231]
Train: [      0       1       2 ... 1775315 1775316 1775317] Test: [ 562975  562992  563216 ... 1066527 1066528 1066529]
Train: [      0       1       2 ... 1775315 1775316 1775317] Test: [ 913555  913679  913781 ... 1549977 1550291 1550807]
Train: [      0       1       2 ... 1549977 1550291 1550807] Test: [1420136 1420137 1420138 ... 1775315 1775316 1775317]
----------------------------------------------------------------------------------------------------
Label Distributions: 

[0.99296253 0.00703747]
[0.99296181 0.00703819]


In [6]:
print('Length of X (train): {} | Length of y (train): {}'.format(len(original_Xtrain), len(original_ytrain)))
print('Length of X (test): {} | Length of y (test): {}'.format(len(original_Xtest), len(original_ytest)))
print("Before UnderSampling, counts of label '1': {}".format(sum(original_ytrain == 1)))
print("Before UnderSampling, counts of label '0': {} \n".format(sum(original_ytrain == 0)))

Length of X (train): 1420255 | Length of y (train): 1420255
Length of X (test): 355063 | Length of y (test): 355063
Before UnderSampling, counts of label '1': 9995
Before UnderSampling, counts of label '0': 1410260 



In [7]:
print("test, counts of label '1': {}".format(sum(original_ytest == 1)))
print("test, counts of label '0': {} \n".format(sum(original_ytest == 0)))

test, counts of label '1': 2499
test, counts of label '0': 352564 



In [8]:
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split, RandomizedSearchCV

sm1 = NearMiss(sampling_strategy={0:500,1:500}) # 500:500
Xsm_train, ysm_train = sm1.fit_resample(original_Xtrain, original_ytrain)
sm2 = NearMiss(sampling_strategy={0:200,1:67}) # 200:200
Xsm_test, ysm_test = sm2.fit_resample(original_Xtest, original_ytest)

In [9]:
print('After Sampling, the shape of train_X: {}'.format(Xsm_train.shape))
print('After Sampling, the shape of train_y: {} \n'.format(ysm_train.shape))

print("After Sampling, counts of label '1': {}".format(sum(ysm_train == 1)))
print("After Sampling, counts of label '0': {}".format(sum(ysm_train == 0)))

After Sampling, the shape of train_X: (1000, 14)
After Sampling, the shape of train_y: (1000,) 

After Sampling, counts of label '1': 500
After Sampling, counts of label '0': 500


In [10]:
Xsm_train_df, ysm_train_df = pd.DataFrame(Xsm_train), pd.DataFrame(ysm_train)
Xsm_train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.881832,-1.584418,0.344253,-0.43232,-0.571375,-0.947434,-0.665307,-0.105932,-0.084376,0.0,0.0,0.0,0.0,1.0
1,0.881832,-1.584418,0.344253,-0.43232,-0.571375,-0.947434,-0.665307,-0.105932,-0.084376,0.0,0.0,0.0,0.0,1.0
2,0.881832,-1.584418,0.344253,-0.43232,-0.571375,-0.947434,-0.665307,-0.105932,-0.084376,0.0,0.0,0.0,0.0,1.0
3,0.179825,-1.231264,0.344253,-0.43232,-0.571375,-0.661681,-0.665307,-0.105932,-0.084376,0.0,0.0,0.0,0.0,1.0
4,0.179825,-1.231264,0.344253,-0.43232,-0.571375,-0.661681,-0.665307,-0.105932,-0.084376,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-1.838447,-1.584418,-1.488533,-0.43232,0.213868,-0.947434,-0.665307,2.715014,-0.084376,0.0,0.0,0.0,1.0,1.0
996,0.837957,-1.024682,0.344253,-0.43232,0.302822,-0.041619,-0.665307,-0.105932,-0.084376,0.0,0.0,0.0,0.0,1.0
997,-0.478307,-0.671527,0.344253,-0.43232,0.378032,0.468242,-0.665307,-0.105932,-0.084376,0.0,0.0,0.0,0.0,1.0
998,-2.847583,-1.584418,-1.488533,-0.43232,0.443182,-0.661681,-0.665307,-0.105932,-0.084376,0.0,0.0,0.0,0.0,1.0


In [17]:
pca = PCA(.96)
pca.fit(Xsm_train_df)
pca.n_components_

8

In [18]:
original_Xtest_df = pd.DataFrame(Xsm_test)

In [19]:
Xsm_train_df_pca = pd.DataFrame(pca.transform(Xsm_train_df))
original_Xtest_df_pca = pd.DataFrame(pca.transform(original_Xtest_df))

In [20]:
Xsm_train_df_pca.to_csv('Xmiss_train_pca.csv',header=False,index=False)
ysm_train_df.to_csv('ymiss_train_pca.csv',header=False,index=False)

In [21]:
qtest_df_pca = pd.concat([original_Xtest_df_pca, pd.DataFrame(ysm_test)], axis=1)
qtest_df_pca.to_csv('qtest_df_pca.csv',header=False,index=False)

In [22]:
import tensorflow as tf

In [25]:
len(data_combined)/5

200.0

In [31]:
"""Fitting script"""
import numpy as np
import os

import tensorflow as tf
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
#tf.compat.v1.disable_eager_execution()

import strawberryfields as sf
from strawberryfields.ops import Dgate, BSgate, Kgate, Sgate, Rgate

import sys
sys.path.append("..")

# ===================================================================================
#                                   Hyperparameters
# ===================================================================================

# Two modes required: one for "genuine" transactions and one for "fradulent"
mode_number = 2
# Number of photonic quantum layers
depth = 4

# Fock basis truncation
cutoff = 10
# Number of batches in optimization
reps = 30000 #30000

# Label for simulation
simulation_label = 1

# Number of batches to use in the optimization
batch_size = 5 #factor of the total rows # it was 24

# Random initialization of gate parameters
sdev_photon = 0.1
sdev = 1

# Variable clipping values
disp_clip = 5
sq_clip = 5
kerr_clip = 1

# If loading from checkpoint, previous batch number reached
ckpt_val = 0

# Number of repetitions between each output to TensorBoard
tb_reps = 100
# Number of repetitions between each model save
savr_reps = 1000

model_string = str(simulation_label)

# Target location of output
folder_locator = './outputs/'

# Locations of TensorBoard and model save outputs
board_string = folder_locator + 'tensorboard/' + model_string + '/'
checkpoint_string = folder_locator + 'models/' + model_string + '/'

# ===================================================================================
#                                   Loading the training data
# ===================================================================================

# Combining genuine and fraudulent data
data_Xsm_train = np.loadtxt('Xmiss_train_pca.csv', delimiter=',')
data_ysm_train = np.loadtxt('ymiss_train_pca.csv', delimiter=',')

# Combining genuine and fraudulent data
data_combined = np.column_stack([data_Xsm_train, data_ysm_train])
data_points = len(data_combined)
print(data_combined)
# ===================================================================================
#                                   Setting up the classical NN input
# ===================================================================================

# Input neurons
input_neurons = 8  ## number of features
# Widths of hidden layers
nn_architecture = [10, 10] 
# Output neurons of classical part
output_neurons = 14

# Defining classical network parameters
#input_classical_layer = tf.placeholder(tf.float32, shape=[batch_size, input_neurons])
input_classical_layer = tf.placeholder(tf.float32, shape=[batch_size, input_neurons])

layer_matrix_1 = tf.Variable(tf.random_normal(shape=[input_neurons, nn_architecture[0]]))
offset_1 = tf.Variable(tf.random_normal(shape=[nn_architecture[0]]))

layer_matrix_2 = tf.Variable(tf.random_normal(shape=[nn_architecture[0], nn_architecture[1]]))
offset_2 = tf.Variable(tf.random_normal(shape=[nn_architecture[1]]))

layer_matrix_3 = tf.Variable(tf.random_normal(shape=[nn_architecture[1], output_neurons]))
offset_3 = tf.Variable(tf.random_normal(shape=[output_neurons]))

# Creating hidden layers and output
layer_1 = tf.nn.elu(tf.matmul(input_classical_layer, layer_matrix_1) + offset_1)
layer_2 = tf.nn.elu(tf.matmul(layer_1, layer_matrix_2) + offset_2)

output_layer = tf.nn.elu(tf.matmul(layer_2, layer_matrix_3) + offset_3)

# ===================================================================================
#                                   Defining QNN parameters
# ===================================================================================

# Number of beamsplitters in interferometer
bs_in_interferometer = int(1.0 * mode_number * (mode_number - 1) / 2)

with tf.name_scope('variables'):
    bs_variables = tf.Variable(tf.random_normal(shape=[depth, bs_in_interferometer, 2, 2]
                                                , stddev=sdev))
    phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number, 2], stddev=sdev))

    sq_magnitude_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                          , stddev=sdev_photon))
    sq_phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                      , stddev=sdev))
    disp_magnitude_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                            , stddev=sdev_photon))
    disp_phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                        , stddev=sdev))
    kerr_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number], stddev=sdev_photon))

parameters = [layer_matrix_1, offset_1, layer_matrix_2, offset_2, layer_matrix_3, offset_3, bs_variables,
              phase_variables, sq_magnitude_variables, sq_phase_variables, disp_magnitude_variables,
              disp_phase_variables, kerr_variables]


# ===================================================================================
#                                   Constructing quantum layers
# ===================================================================================


# Defining input QNN layer, whose parameters are set by the outputs of the classical network
def input_qnn_layer():
    with tf.name_scope('inputlayer'):
        Sgate(tf.clip_by_value(output_layer[:, 0], -sq_clip, sq_clip), output_layer[:, 1]) | q[0]
        Sgate(tf.clip_by_value(output_layer[:, 2], -sq_clip, sq_clip), output_layer[:, 3]) | q[1]

        BSgate(output_layer[:, 4], output_layer[:, 5]) | (q[0], q[1])

        Rgate(output_layer[:, 6]) | q[0]
        Rgate(output_layer[:, 7]) | q[1]

        Dgate(tf.clip_by_value(output_layer[:, 8], -disp_clip, disp_clip), output_layer[:, 9]) \
        | q[0]
        Dgate(tf.clip_by_value(output_layer[:, 10], -disp_clip, disp_clip), output_layer[:, 11]) \
        | q[1]

        Kgate(tf.clip_by_value(output_layer[:, 12], -kerr_clip, kerr_clip)) | q[0]
        Kgate(tf.clip_by_value(output_layer[:, 13], -kerr_clip, kerr_clip)) | q[1]


# Defining standard QNN layers
def qnn_layer(layer_number):
    with tf.name_scope('layer_{}'.format(layer_number)):
        BSgate(bs_variables[layer_number, 0, 0, 0], bs_variables[layer_number, 0, 0, 1]) \
        | (q[0], q[1])

        for i in range(mode_number):
            Rgate(phase_variables[layer_number, i, 0]) | q[i]

        for i in range(mode_number):
            Sgate(tf.clip_by_value(sq_magnitude_variables[layer_number, i], -sq_clip, sq_clip),
                  sq_phase_variables[layer_number, i]) | q[i]

        BSgate(bs_variables[layer_number, 0, 1, 0], bs_variables[layer_number, 0, 1, 1]) \
        | (q[0], q[1])

        for i in range(mode_number):
            Rgate(phase_variables[layer_number, i, 1]) | q[i]

        for i in range(mode_number):
            Dgate(tf.clip_by_value(disp_magnitude_variables[layer_number, i], -disp_clip,
                                   disp_clip), disp_phase_variables[layer_number, i]) | q[i]

        for i in range(mode_number):
            Kgate(tf.clip_by_value(kerr_variables[layer_number, i], -kerr_clip, kerr_clip)) | q[i]


# ===================================================================================
#                                   Defining QNN
# ===================================================================================

# construct the two-mode Strawberry Fields engine
eng, q = sf.Engine(mode_number)

# construct the circuit
with eng:
    input_qnn_layer()
    for i in range(depth):
        qnn_layer(i)

# run the engine (in batch mode)
state = eng.run("tf", cutoff_dim=cutoff, eval=False, batch_size=batch_size)
# extract the state
ket = state.ket()


# ===================================================================================
#                                   Setting up cost function
# ===================================================================================

# Classifications for whole batch: rows act as data points in the batch and columns
# are the one-hot classifications
classification = tf.placeholder(shape=[batch_size, 2], dtype=tf.int32)

func_to_minimise = 0

# Building up the function to minimize by looping through batch
for i in range(batch_size):
    # Probabilities corresponding to a single photon in either mode
    prob = tf.abs(ket[i, classification[i, 0], classification[i, 1]]) ** 2
    # These probabilities should be optimised to 1
    func_to_minimise += (1.0 / batch_size) * (prob - 1) ** 2

# Defining the cost function
cost_func = func_to_minimise
tf.summary.scalar('Cost', cost_func)

# ===================================================================================
#                                   Training
# ===================================================================================

# We choose the Adam optimizer
optimiser = tf.train.AdamOptimizer()
training = optimiser.minimize(cost_func)

# Saver/Loader for outputting model
saver = tf.train.Saver(parameters)

session = tf.Session()
session.run(tf.global_variables_initializer())

# Load previous model if non-zero ckpt_val is specified
if ckpt_val != 0:
    saver.restore(session, checkpoint_string + 'sess.ckpt-' + str(ckpt_val))

# TensorBoard writer
writer = tf.summary.FileWriter(board_string)
merge = tf.summary.merge_all()

counter = ckpt_val

# Tracks optimum value found (set high so first iteration encodes value)
opt_val = 1e20
# Batch number in which optimum value occurs
opt_position = 0
# Flag to detect if new optimum occured in last batch
new_opt = False
print('done1')
while counter <= reps:
    # Shuffles data to create new epoch
    np.random.shuffle(data_combined)

    # Splits data into batches    
    split_data = np.split(data_combined, data_points / batch_size) #equal divisions, array_split->nearly equal divisions

    for batch in split_data:
        # there are arrays with shape(40,14) which can't be reshaped into (40,14). So, filtering (40,14) arrays
        if counter > reps:
            break
        # Input data (provided as principal components)
        data_points_principal_components = batch[:, :input_neurons] #batch[:, 1:input_neurons+1] if there are indices in col1
        #data_points_principal_components = data_points_principal_components.astype(np.float32)
        
        # Data classes
        classes = batch[:, -1]

        # Encoding classes into one-hot form
        one_hot_input = np.zeros((batch_size, 2), np.int32)

        for i in range(batch_size):
            if int(classes[i]) == 0:
                # Encoded such that genuine transactions should be outputted as a photon in the first mode
                one_hot_input[i] = [1, 0]
            else:
                one_hot_input[i] = [0, 1]

        #one_hot_input = one_hot_input.astype(np.int32)

        # Output to TensorBoard
        if counter % tb_reps == 0:
            [summary, training_run, func_to_minimise_run] = session.run([merge, training, func_to_minimise],
                                                                        feed_dict={
                                                                            input_classical_layer:
                                                                                data_points_principal_components,
                                                                            classification: one_hot_input})
            writer.add_summary(summary, counter)

        else:
            # Standard run of training
            [training_run, func_to_minimise_run] = session.run([training, func_to_minimise], feed_dict={
                input_classical_layer: data_points_principal_components, classification: one_hot_input})

        # Ensures cost function is well behaved
        if np.isnan(func_to_minimise_run):
            compute_grads = session.run(optimiser.compute_gradients(cost_func),
                                        feed_dict={input_classical_layer: data_points_principal_components,
                                                   classification: one_hot_input})
            if not os.path.exists(checkpoint_string):
                os.makedirs(checkpoint_string)
            # If cost function becomes NaN, output value of gradients for investigation
            np.save(checkpoint_string + 'NaN.npy', compute_grads)
            print('NaNs outputted - leaving at step ' + str(counter))
            raise SystemExit

        # Test to see if new optimum found in current batch
        if func_to_minimise_run < opt_val:
            opt_val = func_to_minimise_run
            opt_position = counter
            new_opt = True

        # Save model every fixed number of batches, provided a new optimum value has occurred
        if (counter % savr_reps == 0) and (i != 0) and new_opt and (not np.isnan(func_to_minimise_run)):
            if not os.path.exists(checkpoint_string):
                os.makedirs(checkpoint_string)
            saver.save(session, checkpoint_string + 'sess.ckpt', global_step=counter)
            # Saves position of optimum and corresponding value of cost function
            np.savetxt(checkpoint_string + 'optimum.txt', [opt_position, opt_val])

        counter += 1


[[-1.32931412  0.1654671  -0.21998954 ... -0.02012723  0.00725466
   0.        ]
 [-1.32931412  0.1654671  -0.21998954 ... -0.02012723  0.00725466
   0.        ]
 [-1.32931412  0.1654671  -0.21998954 ... -0.02012723  0.00725466
   0.        ]
 ...
 [-0.03520317  0.7272464   0.62547005 ...  0.17382669 -0.41565043
   1.        ]
 [ 2.83593721  0.24738456 -0.13918545 ... -0.09135539  0.05837374
   1.        ]
 [-1.41839267  0.13190045 -0.14533605 ... -0.01848664  0.15389684
   1.        ]]
done1


In [41]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

[]

In [40]:
tf.reset_default_graph()

In [38]:
267/3

89.0

In [42]:
# Copyright 2018 Xanadu Quantum Technologies Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Fraud detection fitting script"""
import numpy as np
import os

import tensorflow as tf

import strawberryfields as sf
from strawberryfields.ops import Dgate, BSgate, Kgate, Sgate, Rgate

import sys
sys.path.append("..")

# ===================================================================================
#                                   Hyperparameters
# ===================================================================================

# Two modes required: one for "genuine" transactions and one for "fradulent"
mode_number = 2
# Number of photonic quantum layers
depth = 4

# Fock basis truncation
cutoff = 10

# Label for simulation
simulation_label = 1

# Random initialization of gate parameters
sdev_photon = 0.1
sdev = 1

# Variable clipping values
disp_clip = 5
sq_clip = 5
kerr_clip = 1

# If loading from checkpoint, previous batch number reached
ckpt_val = 30000 #30000

model_string = str(simulation_label)

# Target location of output
folder_locator = './outputs/'

# Locations of model saves and where confusion matrix will be saved
checkpoint_string = folder_locator + 'models/' + model_string + '/'
confusion_string = folder_locator + 'confusion/' + model_string + '/'

# ===================================================================================
#                                   Loading the testing data
# ===================================================================================

# Loading combined dataset with extra genuine datapoints unseen in training
data_combined = np.loadtxt('qtest_df_pca.csv', delimiter=',')[:]

# Set to a size so that the data can be equally split up with no remainder
batch_size = 3 #constant for qtest_df data

data_combined_points = len(data_combined)

# ===================================================================================
#                                   Setting up the classical NN input
# ===================================================================================

# Input neurons
input_neurons = 8
# Widths of hidden layers
nn_architecture = [10, 10]
# Output neurons of classical part
output_neurons = 14

# Defining classical network parameters
input_classical_layer = tf.placeholder(tf.float32, shape=[batch_size, input_neurons])

layer_matrix_1 = tf.Variable(tf.random_normal(shape=[input_neurons, nn_architecture[0]]))
offset_1 = tf.Variable(tf.random_normal(shape=[nn_architecture[0]]))

layer_matrix_2 = tf.Variable(tf.random_normal(shape=[nn_architecture[0], nn_architecture[1]]))
offset_2 = tf.Variable(tf.random_normal(shape=[nn_architecture[1]]))

layer_matrix_3 = tf.Variable(tf.random_normal(shape=[nn_architecture[1], output_neurons]))
offset_3 = tf.Variable(tf.random_normal(shape=[output_neurons]))

# Creating hidden layers and output
layer_1 = tf.nn.elu(tf.matmul(input_classical_layer, layer_matrix_1) + offset_1)
layer_2 = tf.nn.elu(tf.matmul(layer_1, layer_matrix_2) + offset_2)

output_layer = tf.nn.elu(tf.matmul(layer_2, layer_matrix_3) + offset_3)

# ===================================================================================
#                                   Defining QNN parameters
# ===================================================================================

# Number of beamsplitters in interferometer
bs_in_interferometer = int(1.0 * mode_number * (mode_number - 1) / 2)

with tf.name_scope('variables'):
    bs_variables = tf.Variable(tf.random_normal(shape=[depth, bs_in_interferometer, 2, 2]
                                                , stddev=sdev))
    phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number, 2], stddev=sdev))

    sq_magnitude_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                          , stddev=sdev_photon))
    sq_phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                      , stddev=sdev))
    disp_magnitude_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                            , stddev=sdev_photon))
    disp_phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                        , stddev=sdev))
    kerr_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number], stddev=sdev_photon))

parameters = [layer_matrix_1, offset_1, layer_matrix_2, offset_2, layer_matrix_3, offset_3, bs_variables,
              phase_variables, sq_magnitude_variables, sq_phase_variables, disp_magnitude_variables,
              disp_phase_variables, kerr_variables]


# ===================================================================================
#                                   Constructing quantum layers
# ===================================================================================


# Defining input QNN layer, whose parameters are set by the outputs of the classical network
def input_qnn_layer():
    with tf.name_scope('inputlayer'):
        Sgate(tf.clip_by_value(output_layer[:, 0], -sq_clip, sq_clip), output_layer[:, 1]) | q[0]
        Sgate(tf.clip_by_value(output_layer[:, 2], -sq_clip, sq_clip), output_layer[:, 3]) | q[1]

        BSgate(output_layer[:, 4], output_layer[:, 5]) | (q[0], q[1])

        Rgate(output_layer[:, 6]) | q[0]
        Rgate(output_layer[:, 7]) | q[1]

        Dgate(tf.clip_by_value(output_layer[:, 8], -disp_clip, disp_clip), output_layer[:, 9]) \
        | q[0]
        Dgate(tf.clip_by_value(output_layer[:, 10], -disp_clip, disp_clip), output_layer[:, 11]) \
        | q[1]

        Kgate(tf.clip_by_value(output_layer[:, 12], -kerr_clip, kerr_clip)) | q[0]
        Kgate(tf.clip_by_value(output_layer[:, 13], -kerr_clip, kerr_clip)) | q[1]


# Defining standard QNN layers
def qnn_layer(layer_number):
    with tf.name_scope('layer_{}'.format(layer_number)):
        BSgate(bs_variables[layer_number, 0, 0, 0], bs_variables[layer_number, 0, 0, 1]) \
        | (q[0], q[1])

        for i in range(mode_number):
            Rgate(phase_variables[layer_number, i, 0]) | q[i]

        for i in range(mode_number):
            Sgate(tf.clip_by_value(sq_magnitude_variables[layer_number, i], -sq_clip, sq_clip),
                  sq_phase_variables[layer_number, i]) | q[i]

        BSgate(bs_variables[layer_number, 0, 1, 0], bs_variables[layer_number, 0, 1, 1]) \
        | (q[0], q[1])

        for i in range(mode_number):
            Rgate(phase_variables[layer_number, i, 1]) | q[i]

        for i in range(mode_number):
            Dgate(tf.clip_by_value(disp_magnitude_variables[layer_number, i], -disp_clip,
                                   disp_clip), disp_phase_variables[layer_number, i]) | q[i]

        for i in range(mode_number):
            Kgate(tf.clip_by_value(kerr_variables[layer_number, i], -kerr_clip, kerr_clip)) | q[i]


# ===================================================================================
#                                   Defining QNN
# ===================================================================================

# construct the two-mode Strawberry Fields engine
eng, q = sf.Engine(mode_number)

# construct the circuit
with eng:
    input_qnn_layer()

    for i in range(depth):
        qnn_layer(i)

# run the engine (in batch mode)
state = eng.run("tf", cutoff_dim=cutoff, eval=False, batch_size=batch_size)
# extract the state
ket = state.ket()

# ===================================================================================
#                                   Extracting probabilities
# ===================================================================================

# Classifications for whole batch: rows act as data points in the batch and columns
# are the one-hot classifications
classification = tf.placeholder(shape=[batch_size, 2], dtype=tf.int32)

prob = []

for i in range(batch_size):
    # Finds the probability of a photon being in either mode
    prob.append([tf.abs(ket[i, 1, 0]) ** 2, tf.abs(ket[i, 0, 1]) ** 2])

# ===================================================================================
#                                   Testing performance
# ===================================================================================

# Defining array of thresholds from 0 to 1 to consider in the ROC curve
thresholds_points = 101
thresholds = np.linspace(0, 1, num=thresholds_points)

# Saver/Loader for outputting model
saver = tf.train.Saver(parameters)

session = tf.Session()
session.run(tf.global_variables_initializer())

saver.restore(session, checkpoint_string + 'sess.ckpt-' + str(ckpt_val))
print('done1')

# Split up data to process in batches
data_split = np.split(data_combined, data_combined_points / batch_size)

# Defining confusion table
confusion_table = np.zeros((thresholds_points, 2, 2))

for batch in data_split:
    # Input data (provided as principal components)
    data_points_principal_components = batch[:, 1:input_neurons + 1]
    data_points_principal_components = data_points_principal_components.astype(np.float32)
    # Data classes
    classes = batch[:, -1]
    classes = classes.astype(np.int32)
    # Probabilities outputted from circuit
    prob_run = session.run(prob, feed_dict={input_classical_layer: data_points_principal_components})

    for i in range(batch_size):
        # Calculate probabilities of photon coming out of either mode
        p = prob_run[i]
        # Normalize to these two events (i.e. ignore all other outputs)
        p = p / np.sum(p)

        # Predicted class is a list corresponding to threshold probabilities
        predicted_class = []

        for j in range(thresholds_points):
            # If probability of a photon exiting first mode is larger than threshold, attribute as genuine
            if p[0] > thresholds[j]:
                predicted_class.append(0)
            else:
                predicted_class.append(1)

        actual_class = classes[i]

        # Constructing confusion table
        for j in range(2):
            for k in range(2):
                for l in range(thresholds_points):
                    if actual_class == j and predicted_class[l] == k:
                        confusion_table[l, j, k] += 1

# Renormalizing confusion table
for i in range(thresholds_points):
    confusion_table[i] = confusion_table[i] / data_combined_points * 100

if not os.path.exists(confusion_string):
    os.makedirs(confusion_string)

# Save as numpy array
np.save(confusion_string + 'confusion_table.npy', confusion_table)


INFO:tensorflow:Restoring parameters from ./outputs/models/1/sess.ckpt-30000
done1
